In [3]:
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler

n = 16
q = 32768
t = 256
sigma = 8
delta = q//t

S.<x> = PolynomialRing(ZZ)
R.<zeta> = S.quotient(S.ideal(x**(n) + 1, q))

SampleSmallPoly = lambda :  R(DiscreteGaussianDistributionPolynomialSampler(ZZ['x'], n, sigma)())

def round_to_nearest(x, delta):
    # a // b = floor(a/b), where RHS is rational division
    # (a + (b//2))//b ~ floor(a/b + 1/2) = round_to_nearest_int(a/b)
    return (x + delta//2) // delta

def poly_round(poly, delta):
    return R(lift(poly).map_coefficients(lambda c: round_to_nearest(c, delta)))


In [54]:
def keygen():
    return SampleSmallPoly()

def enc(m, s):
    a = R.random_element()
    e = SampleSmallPoly()
    print("a = ", a)
    print("s =", s)
    print("as + e + delta*m = ", a*s + e + delta*m)
    return (a, a*s + e + delta*m)

def dec(ct, s):
    (a, b) = ct
    partial_dec = b - a * s
    return R(partial_dec.lift().map_coefficients(lambda c: round_to_nearest(c, delta)))

def hom_sum(ct0, ct1):
    (a0, b0) = ct0
    (a1, b1) = ct1
    return (a0+a1, b0+b1)

def hom_prod_and_dec(ct0, ct1, s):
    (u0, v0) = ct0
    (u1, v1) = ct1
    num = v0*v1 - s * (u0*v1+v0*u1) + s*s*u0*u1
    print("v0*v1 = ", v0*v1)
    print( "s * (u0*v1+v0*u1) = ", s * (u0*v1+v0*u1))
    print("s*s*u0*u1 = ", s*s*u0*u1)
    print("num = ", num)
    print("num.parent() = ",num.parent())
    print("delta^2 = ", delta**2)
    print("q =", q)
    return poly_round(num, delta**2)
    

In [150]:
s = keygen()
m0 = 3
m1 = 2
ct0 = enc(m0, s)
ct1 = enc(m1, s)
#debugging prints
(u0, v0) = ct0
(u1, v1) = ct1
print(u0.parent())
print(u1.parent())
print("u0 = ", u0)
print("v0 = ", v0)
print("u1 = ", u1)
print("v1 = ", v1)
hom_prod_and_dec(ct0, ct1, s)

a =  2*zeta + 1
s = 4*zeta^15 - 6*zeta^14 - 7*zeta^13 + 4*zeta^12 + 5*zeta^11 - 13*zeta^10 - 3*zeta^9 - 7*zeta^8 - zeta^7 + zeta^6 + 9*zeta^5 + 5*zeta^4 - 19*zeta^3 - 8*zeta^2 + zeta - 13
as + e + delta*m =  8*zeta^16 - 17*zeta^15 - 10*zeta^14 + 6*zeta^13 + 18*zeta^12 - 14*zeta^11 - 24*zeta^10 - 7*zeta^9 - 9*zeta^8 + 2*zeta^7 + 40*zeta^6 + 28*zeta^5 - 27*zeta^4 - 13*zeta^3 - 8*zeta^2 - 26*zeta + 363
a =  zeta^2 + zeta + 2
s = 4*zeta^15 - 6*zeta^14 - 7*zeta^13 + 4*zeta^12 + 5*zeta^11 - 13*zeta^10 - 3*zeta^9 - 7*zeta^8 - zeta^7 + zeta^6 + 9*zeta^5 + 5*zeta^4 - 19*zeta^3 - 8*zeta^2 + zeta - 13
as + e + delta*m =  4*zeta^17 - 2*zeta^16 - 16*zeta^15 - 19*zeta^14 - 7*zeta^13 + 10*zeta^12 - 9*zeta^11 - 48*zeta^10 - 13*zeta^9 - 19*zeta^8 + 6*zeta^7 + 18*zeta^6 + 4*zeta^5 - 16*zeta^4 - 57*zeta^3 - 27*zeta^2 - 13*zeta + 237
Quotient of Univariate Polynomial Ring in x over Integer Ring by the ideal (x^16 + 1, 32768)
Quotient of Univariate Polynomial Ring in x over Integer Ring by the ideal (x^16 

6

In [70]:
R.random_element()

5*zeta^2

In [72]:
type(R)

<class 'sage.rings.quotient_ring.QuotientRing_generic_with_category'>

In [73]:
R.random_element??

Signature: R.random_element()
Docstring:
   Return a random element of this quotient ring obtained by sampling
   a random element of the cover ring and reducing it modulo the
   defining ideal.

   EXAMPLES:

      sage: R.<x,y> = QQ[]
      sage: S = R.quotient([x^3, y^2])
      sage: S.random_element()  # random
      -8/5*xbar^2 + 3/2*xbar*ybar + 2*xbar - 4/23
Source:   
    def random_element(self):
        r"""
        Return a random element of this quotient ring obtained by
        sampling a random element of the cover ring and reducing
        it modulo the defining ideal.

        EXAMPLES::

            sage: R.<x,y> = QQ[]
            sage: S = R.quotient([x^3, y^2])
            sage: S.random_element()  # random
            -8/5*xbar^2 + 3/2*xbar*ybar + 2*xbar - 4/23

        TESTS:

        Make sure we are not just getting images of integers in this
        ring (which would be the case if the default implementation
        of this method was inherited from generic ring

In [157]:
[S.random_element() for _ in range(100)]

[-x^2 - 3*x,
 -x^2 - 2*x + 2,
 3*x + 1,
 x^2 - x + 1,
 4*x + 1,
 -x^2 - x + 2,
 -x^2 + 2,
 -2,
 -3,
 -x^2 - 3*x + 1,
 -5*x^2 - 4*x,
 4,
 -2*x^2 - x - 1,
 -3*x^2 - 2*x,
 x^2 - x - 1,
 8*x^2 + 9*x,
 2*x^2 + 6*x,
 x^2 + x,
 -x^2 + 3*x - 1,
 3*x^2 - 5*x,
 -2*x^2,
 -3*x - 1,
 -x^2 + 9*x - 1,
 -3*x^2 - 2*x + 2,
 2*x^2 + 3,
 12*x^2 + x + 1,
 -x^2 - 2*x,
 -2*x^2 + 2*x + 1,
 -6*x^2 - x - 1,
 328*x^2 + 1,
 -4*x^2 - x + 10,
 -2*x^2 + x - 1,
 4*x^2 + 11*x - 1,
 -x^2 - 1,
 -8*x^2 + 6*x + 3,
 -2*x^2 + 2*x - 17,
 -x^2 - x,
 4*x^2 + x,
 x^2 + x + 1,
 x^2 + 2*x - 1,
 -153*x^2 + 2*x + 4,
 -5*x + 2,
 29*x^2 - 9*x - 1,
 -2*x - 2,
 -x^2 - 1,
 -x^2 - 28,
 42*x^2 + 3*x - 1,
 6*x^2,
 -x^2 - 1,
 -4*x^2 + 1,
 -2*x + 1,
 x^2 - 2*x - 1,
 -x^2 - 7*x + 2,
 x^2 - 5*x + 1,
 x^2 + x,
 -10*x + 1,
 x^2 - 3*x,
 -x^2 - x - 2,
 -16*x - 4,
 x^2 + x - 1,
 -1,
 -4*x^2 - x + 2,
 -5*x^2 + x + 1,
 2*x - 1,
 6*x^2 - 1,
 7*x + 1,
 -x^2 - 5*x - 1,
 -x^2 - 1,
 x^2 + x + 1,
 x,
 -3*x^2,
 6*x^2 - 1,
 2*x + 1,
 x^2 - 2*x,
 -x^2 + x - 1

In [102]:
S.random_element??

Signature: S.random_element(degree=(-1, 2), monic=False, *args, **kwds)
Docstring:
   Return a random polynomial of given degree (bounds).

   INPUT:

   * "degree" -- (default: "(-1, 2)") integer for fixing the degree or
     a tuple of minimum and maximum degrees

   * "monic" -- boolean (default: "False"); indicate whether the
     sampled polynomial should be monic

   * "*args, **kwds" -- additional keyword parameters passed on to the
     "random_element" method for the base ring

   EXAMPLES:

      sage: R.<x> = ZZ[]
      sage: f = R.random_element(10, x=5, y=10)
      sage: f.degree()
      10
      sage: f.parent() is R
      True
      sage: all(a in range(5, 10) for a in f.coefficients())
      True
      sage: R.random_element(6).degree()
      6

   If a tuple of two integers is given for the "degree" argument, a
   polynomial is chosen among all polynomials with degree between
   them. If the base ring can be sampled uniformly, then this method
   also samples uniformly

In [117]:
ZZ.random_element??

Docstring:
   Return a random integer.

   INPUT:

   * "x", "y" integers -- bounds for the result

   * "distribution" -- string:

     * "'uniform'"

     * "'mpz_rrandomb'"

     * "'1/n'"

     * "'gaussian'"

   OUTPUT: with no input, return a random integer

      If only one integer x is given, return an integer between 0 and
      x-1.

      If two integers are given, return an integer between x and y-1
      inclusive.

      If at least one bound is given, the default distribution is the
      uniform distribution; otherwise, it is the distribution
      described below.

      If the distribution "'1/n'" is specified, the bounds are
      ignored.

      If the distribution "'mpz_rrandomb'" is specified, the output is
      in the range from 0 to 2^x - 1.

      If the distribution "'gaussian'" is specified, the output is
      sampled from a discrete Gaussian distribution with parameter
      \sigma=x and centered at zero. That is, the integer v is
      returned with prob

In [168]:
ZZ.random_element()

2